In [4]:
import time
import requests
from pprint import pprint
import os

cfg = {}
cfg['imageRoot'] = 'D:/Dev/workspace/traffic/map-tile/nanjing/dark/'
cfg['pngFiles'] = cfg['imageRoot'] + 'img/'
cfg['defaultSeaPng'] = cfg['imageRoot'] + 'defaults/sea.png'
cfg['defaultLandPng'] = cfg['imageRoot'] + 'defaults/land.png'
cfg['metaDir'] = cfg['imageRoot'] + '/meta/'

cfg['outDefaultCsv'] = cfg['imageRoot'] + '/csv/map_tile_default.csv'
cfg['outTileCsv'] = cfg['imageRoot'] + '/csv/map_tile.csv'

In [5]:
def readBinFile(pathname):
    data = None
    statinfo = os.stat(pathname)
    if statinfo.st_size <= 0:
        raise Exception('Invalid file: ' + pathname)
    with open(pathname, 'rb') as f:
        data = f.read(statinfo.st_size)
    return ''.join('{:02x}'.format(x) for x in data)

# print(readBinFile(cfg['defaultSeaPng']))

In [6]:
def generateDefaultCsv():
    with open(cfg['outDefaultCsv'], 'w') as f:
        f.write('THEME,PNG,TAG\n')
        f.write(','.join(['dark', readBinFile(cfg['defaultLandPng']), 'land\n']))
        f.write(','.join(['dark', readBinFile(cfg['defaultSeaPng']), 'sea\n']))

generateDefaultCsv()

In [20]:
import csv

def generateTileCsv():
    with open(cfg['outTileCsv'], 'w') as f:
        # head line
        f.write('THEME,Z,X,Y,PNG,TAG\n')

        # for png tiles
        for root, subdirs, files in os.walk(cfg['pngFiles']):
            #print(root, subdirs, files, '\n')
            for file in files:
                if file.endswith('.png'):
                    z_str = root[root.rfind('/') + 1:]

                    pathname = os.path.join(root, file)
                    f.write(','.join(['dark',
                                      z_str,
                                      file.replace('-', ',').replace('.png', ''),
                                      readBinFile(pathname),
                                      '\n']))

        # for default land and sea pngs
        for file in os.listdir(cfg['metaDir']):
            if file.endswith('.meta'):
                str_z = file.replace('.meta', '')

                with open(os.path.join(cfg['metaDir'], file), 'r') as csv_file:
                    reader = csv.reader(csv_file)
                    for line in reader:
                        x, y, tag = line[0], line[1], line[2]
                        f.write(','.join(['dark', str_z, str(x), str(y), '',tag + '\n']))

generateTileCsv()